In [28]:
from amadeus import Client, ResponseError
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )

In [29]:
try:
    firstResponse = amadeus.shopping.flight_offers.prediction.post(
        amadeus.shopping.flight_offers_search.get(
        originLocationCode='SYD',
        destinationLocationCode='BKK',
        departureDate='2022-11-14',
        adults=1).result)
except ResponseError as error:
    print(error)

In [30]:
try:
    secondResponse = amadeus.shopping.flight_offers_search.get(
    originLocationCode='MAD',
    destinationLocationCode='BOS',
    departureDate='2022-07-07',
    adults='4'
    )
except ResponseError as error:
    print(error)

In [4]:
print(secondResponse.data[3]['itineraries'][0]['segments'][2])

{'departure': {'iataCode': 'JFK', 'terminal': '5', 'at': '2022-07-08T11:12:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'C', 'at': '2022-07-08T12:28:00'}, 'carrierCode': 'TP', 'number': '4468', 'aircraft': {'code': 'E90'}, 'operating': {'carrierCode': 'B6'}, 'duration': 'PT1H16M', 'id': '224', 'numberOfStops': 0, 'blacklistedInEU': False}


In [23]:
import classes
import datetime
from classes import Transport

class Flights:
    def __init__(self, amadeus_client):
        self.amadeus = amadeus_client
    
    def getFlightObjects(self, srcAirportCode, dstAirportCode, date, passengersCount):
        response = self.amadeus.shopping.flight_offers_search.get(
        originLocationCode=srcAirportCode,
        destinationLocationCode=dstAirportCode,
        departureDate=date,
        adults=passengersCount).data
        parsedData = self.parseFlightResponse(response)
        return parsedData
    
    def extractDuration(self, itineraries):
        hour = 0
        minute = 0
        for i in itineraries:
            i = {x.replace(' ', ''): v
    for x, v in i.items()}
            currDur = i['duration'].split('M')
            if (len(currDur) == 1):
                hour += int(currDur[-1].split('PT')[-1].split('H')[0])
            if (len(currDur) == 2):
                minute += int(currDur[0].split('H')[-1])
                hour += int(currDur[0].split('H')[0].split('PT')[-1])
        return hour + minute / 60
    
    def extractPrice(self, price):
        price = {x.replace(' ', ''): v
             for x, v in price.items()}
        return price['total']
    
    def extractItinerary(self, itineraries):
        flightList = []
        for i in itineraries[0]['segments']:
            print(i)
            timeStart = datetime.datetime.strptime(i['departure']['at'], '%Y-%d-%mT%H:%M:%S')
            timeEnd = datetime.datetime.strptime(i['arrival']['at'], '%Y-%d-%mT%H:%M:%S')
            title = i['carrierCode'] + i['number'] # flight number
            duration = i['duration']
            orderInAdvance = True
            try:
                terminal = i['departure']['terminal']
            except:
                print("No terminal in answer")
                terminal = str(1)
            originAirport = i['departure']['iataCode']
            placeOfOrigin = originAirport + '-' + terminal
            destination = i['arrival']['iataCode']
            methodOfTransport = 5
            cost = 0
            duration = int((timeEnd - timeStart).total_seconds()) / 3600
            currTransportObject = Transport(duration, cost, str(timeStart), str(timeEnd), title, "", "", orderInAdvance, "", 5, "", placeOfOrigin, destination) # consider adding coordinates.
            flightList.append(currTransportObject)
        return flightList
    def flightEndingTimeArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append(i['arrival']['at'])
        return resArr
    
    def flightDestinationLocationArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append({'airport' : i['arrival']['iataCode'],  'terminal' : i['arrival']['terminal']})
        return resArr
    
    def flightStartingTimeArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append(i['departure']['at'])
        return resArr
    
    def flightSourceLocationArray(self, response):
        resArr = []
        for i in response['itineraries'][0]['segments']:
            resArr.append({'airport' : i['departure']['iataCode'], 'tיerminal' : i['departure']['terminal']})
        return resArr
    
    def parseFlightResponse(self, response):
        resultArr = []
        for option in response:
            currResultDict = {}
            option = {x.replace(' ', ''): v
     for x, v in option.items()}
#             currResultDict['duration'] = self.extractDuration(option['itineraries'])
            price = self.extractPrice(option['price'])
            currResultDict['itinerary'] = self.extractItinerary(option['itineraries'])
#             currResultDict['timeStart'] = self.flightStartingTimeArray(option)
#             currResultDict['timeEnd'] = self.flightEndingTimeArray(option)
            itineraryList = self.extractItinerary(option['itineraries'])
            if (len(itineraryList) > 0):
                itineraryList[0].cost = price
            resultArr.append(itineraryList)
        if (len(resultArr) > 0):
            return resultArr[-1]
        return resultArr


In [24]:
from amadeus import Client, ResponseError
amadeus = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
a = Flights(amadeus)
b = a.getFlightObjects('MAD', 'BOS','2022-07-07',  4)

{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-07-07T06:35:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-07-07T06:55:00'}, 'carrierCode': 'TP', 'number': '1023', 'aircraft': {'code': '320'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT1H20M', 'id': '219', 'numberOfStops': 0, 'blacklistedInEU': False}
{'departure': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-07-07T12:00:00'}, 'arrival': {'iataCode': 'EWR', 'terminal': 'B', 'at': '2022-07-07T15:15:00'}, 'carrierCode': 'TP', 'number': '201', 'aircraft': {'code': '32Q'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT8H15M', 'id': '220', 'numberOfStops': 0, 'blacklistedInEU': False}
{'departure': {'iataCode': 'EWR', 'terminal': 'A', 'at': '2022-07-07T19:00:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'C', 'at': '2022-07-07T20:28:00'}, 'carrierCode': 'TP', 'number': '4286', 'aircraft': {'code': 'E90'}, 'operating': {'carrierCode': 'B6'}, 'duration': 'PT1H28M', 'id': '221', 'numberOfSto

In [25]:
print(b[0].methodOfTransportation)

5


In [26]:
def TripToJson(trip):
    transOptions = {0 : "NONE", 1 : "BUS", 2 : "TRAIN", 3 : "RAM", 4 : "PUBLICTAXI", 5 : "FLIGHT"}

    newDays = []
    for day in trip.days:

        newacts = []
        for act in day.activities:
            actson = act.__dict__
            newacts.append(actson)
        day.activities = newacts

        newtrans = []
        for trans in day.transportation:
            transon = trans.__dict__
            transon["methodOfTransportation"] = transOptions[int(transon["methodOfTransportation"])]
            newtrans.append(transon)
        day.transportation = newtrans

        day.placeOfStay = day.placeOfStay.__dict__

        newDays.append(day.__dict__)
    trip.days = newDays
    # print(trip.__dict__)
    trip.userId = str(trip.__dict__["userId"])

    newFlights = []
    for flight in trip.initFlight:
        nFlight = flight.__dict__
        nFlight["methodOfTransportation"] = transOptions.get(int(nFlight["methodOfTransportation"]))
        newFlights.append(nFlight)
    trip.initFlight = newFlights

    newflights2 = []
    for flight in trip.finFlight:
        nFlight = flight.__dict__
        nFlight["methodOfTransportation"] = transOptions.get(int(nFlight["methodOfTransportation"]))
        newflights2.append(nFlight)
    trip.finFlight = newflights2
    return json.dumps(trip.__dict__)

In [27]:
from classes import Trip
import json
c = Trip("MYTRIP XTREME", '', '2', '2022-11-01', '2022-11-10', '', '500', '123', b, [])
print(TripToJson(c))

{"name": "MYTRIP XTREME", "destination": "", "duration": "2", "startDate": "2022-11-01", "endDate": "2022-11-10", "days": [], "cost": "500", "userId": "123", "initFlight": [{"duration": 2.5, "cost": "8987.28", "timeStart": "2022-07-07 11:35:00", "timeEnd": "2022-07-07 14:05:00", "title": "AZ61", "originLink": "", "destinationLink": "", "orderInAdvance": true, "placeOfOrigin": "", "methodOfTransportation": "FLIGHT", "destination": "", "baseStation": "MAD-2", "arrivalStation": "FCO"}, {"duration": 3.3333333333333335, "cost": 0, "timeStart": "2022-08-07 09:50:00", "timeEnd": "2022-08-07 13:10:00", "title": "AZ614", "originLink": "", "destinationLink": "", "orderInAdvance": true, "placeOfOrigin": "", "methodOfTransportation": "FLIGHT", "destination": "", "baseStation": "FCO-3", "arrivalStation": "BOS"}], "finFlight": []}


In [ ]:
def __init__(self, name, destination, duration, startDate, endDate, days, cost, userId, initFlight, finFlight):
        self.name = name
        self.destination = destination
        self.duration = duration
        self.startDate = startDate
        self.endDate = endDate
        self.days = days
        self.cost = cost
        self.userId = userId
        self.initFlight = initFlight
        self.finFlight = finFlight

In [ ]:
print(secondResponse.data[0])
print(secondResponse.data[0].keys())

In [ ]:
# Flight Create Orders
amadeus.booking.flight_orders.post(secondResponse.data[0], ['1'])

In [ ]:
print("DONE")
# Flight Order Management
# The flight ID comes from the Flight Create Orders (in test environment it's temporary)
# Retrieve the order based on it's ID
flight_booking = amadeus.booking.flight_orders.post(body).data
amadeus.booking.flight_order(flight_booking['id']).get()
# Delete the order based on it's ID
amadeus.booking.flight_order(flight_booking['id']).delete()

In [ ]:
class Transport:
    def __init__(self):
        self.duration = int(0)
        self.cost = int(0)
        self.timeStart = datetime.now()
        self.timeEnd = datetime.now()
        self.title = str("")
        self.googleMapsLink = str("")
        self.googleMapsImageLink = str("")
        self.orderInAdvance = bool(False)
        self.placeOfOrigin = str("")
        self.methodOfTransportation =  Transportation.NONE
        self.destination = str("")

In [9]:
import datetime
import classes
from classes import Transportation
from classes import Transport
flightList = []
itineraries = secondResponse.data[3]['itineraries']
#print(itineraries[0])
for i in itineraries[0]['segments']:
    print(i)
    print(i['departure']['at'])
    timeStart = datetime.datetime.strptime(i['departure']['at'], '%Y-%d-%mT%H:%M:%S')
    timeEnd = datetime.datetime.strptime(i['arrival']['at'], '%Y-%d-%mT%H:%M:%S')
    title = i['carrierCode'] + i['number'] # flight number
    duration = i['duration']
    orderInAdvance = True
    placeOfOrigin = i['departure']['iataCode'] + '-' + i['departure']['terminal']
    destination = i['arrival']['iataCode']
    methodOfTransport = Transportation.FLIGHT
    cost = 0
    duration = int((timeEnd - timeStart).total_seconds()) / 3600
    currTransportObject = Transport(duration, cost, timeStart, timeEnd, title, "", "", orderInAdvance, placeOfOrigin, methodOfTransport, destination)
    flightList.append(currTransportObject)
for i in flightList:
    print(i)

{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-07-07T13:35:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-07-07T13:55:00'}, 'carrierCode': 'TP', 'number': '1027', 'aircraft': {'code': '319'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT1H20M', 'id': '222', 'numberOfStops': 0, 'blacklistedInEU': False}
2022-07-07T13:35:00


TypeError: __init__() missing 2 required positional arguments: 'baseStation' and 'arrivalStation'

In [52]:
print(secondResponse.data[25]['itineraries'])

[{'duration': 'PT22H18M', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '2', 'at': '2022-07-07T06:35:00'}, 'arrival': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-07-07T06:55:00'}, 'carrierCode': 'TP', 'number': '1023', 'aircraft': {'code': '320'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT1H20M', 'id': '70', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LIS', 'terminal': '1', 'at': '2022-07-07T12:00:00'}, 'arrival': {'iataCode': 'EWR', 'terminal': 'B', 'at': '2022-07-07T15:15:00'}, 'carrierCode': 'TP', 'number': '201', 'aircraft': {'code': '32Q'}, 'operating': {'carrierCode': 'TP'}, 'duration': 'PT8H15M', 'id': '71', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'EWR', 'terminal': 'C', 'at': '2022-07-07T21:29:00'}, 'arrival': {'iataCode': 'BOS', 'terminal': 'A', 'at': '2022-07-07T22:53:00'}, 'carrierCode': 'TP', 'number': '8510', 'aircraft': {'code': '738'}, 'operating': {'carrierCode': 'UA'}, 'duration